# Definición de CSVs

- Se importa la librería pandas

In [2]:
import pandas as pd

- Se descarga la tabla sp500 de wikipedia.

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

df_sp500 = pd.read_html(url, header=0)[0]


- Se leen los archivos csv con los valores del sp500.

In [4]:
open = pd.read_csv('data/open.csv')
high = pd.read_csv('data/high.csv')
low = pd.read_csv('data/low.csv')
close = pd.read_csv('data/close.csv')
volume = pd.read_csv('data/volume.csv')

- Se define la lista del nombre de sectores.

In [5]:
sector_names = ['Communication Services', 'Consumer Discretionary', 'Consumer Staples', 'Energy',
                'Financials', 'Health Care', 'Industrials', 'Information Technology', 'Materials',
                'Real Estate', 'Utilities']

- Se define la función para extraer los datos combinados por sector.

In [6]:
def df_categorize(data1:pd.DataFrame, data2:pd.DataFrame):
    '''
    :param data1: Dataframe guía de las empresas pertenecientes al S&P
    :param data2: Dataframe extraído de yfinance con todos los valores de tipo "open", "close", "high", "low" y "volume"
    :return: Una serie de dataframes extraídos de "data2" categorizados por el sector en el que se desempeña cada empresa según la columna "GICS Sector" del dataframe "data1".
    '''

    combined = pd.merge(data1, data2.T, left_on='Symbol', right_index=True)

    sector_groups = combined.groupby('GICS Sector')

    sector_dfs = []
    for sector, sector_df in sector_groups:
        sector_dfs.append(sector_df)

    return sector_dfs

- Se define la función que crea una lista de dataframes que elimina las columnas que no serán utilizadas en el proyecto.

In [7]:
def sector_drop(sector_dfs: pd.DataFrame, num:int):
    ''' :param sector_dfs: Lista de dataframes para extraer uno de ellos.
    :param num: Este número define qué dataframe de la lista de dataframe será extraído
    :return: Un dataframe extraído de la lista de dataframes ak que se le han eliminado las columnas que no son relevantes para los pasos siguientes de la extracción de datos. '''
    df_final = sector_dfs[num].T
    df_final.rename(columns=df_final.iloc[0], inplace=True)
    df_final.drop(['Symbol', 'Security', 'GICS Sector', 'GICS Sub-Industry', 'Date added',
                   'Headquarters Location', 'CIK', 'Founded'], inplace=True)

    return df_final

- Se concatenan los dataframes de los valores del SP500 y se define el OHLC de todo el dataframe.

In [9]:
total_ohlc = pd.concat([open, high, low, close])
total_ohlc = (total_ohlc.groupby(by='Date', axis=0).sum()/4)

- Se extrae el OHLC del SP500. y se guarda como csv

In [10]:
spy_df = total_ohlc['SPY']
spy_df.to_csv('data/ohlc_SPY.csv', index=True)

- Se separa el dataframe de OHLC por categorías.

In [11]:
total_ohlc_categorized = df_categorize(df_sp500, total_ohlc)

- Se crea un dataframe del ohlc con cada sector en una lista.

In [12]:
ohlc_sector_data = [sector_drop(total_ohlc_categorized, i) for i in range(len(total_ohlc_categorized))]
ohlc_sector_data = [ohlc_sector_data[i] for i, name in enumerate(sector_names)]

- Se define la lista con los nombres de los archivos que serán guardados en la carpeta data.
- Se guardan los archivos csv con cada dataframe definido anteriormente.

In [13]:
df_names = ['ohlc_communication_services', 'ohlc_consumer_discretionary', 'ohlc_consumer_staples',
            'ohlc_energy', 'ohlc_financials', 'ohlc_health_care', 'ohlc_industrials',
            'ohlc_information_technology', 'ohlc_materials', 'ohlc_real_estate', 'ohlc_utilities']

i=0
for df in ohlc_sector_data:
    df.to_csv(f'data/{df_names[i]}.csv', index=True)
    i+=1

- Se definen los nombres de los promedios.
- Se crea un dataset con los promedios de cada symbol separado por sector.

In [39]:
df_names_avg = ['avg_communication_services', 'avg_consumer_discretionary', 'avg_consumer_staples',
                'avg_energy', 'avg_financials', 'avg_health_care', 'avg_industrials',
                'avg_information_technology', 'avg_materials', 'avg_real_estate', 'avg_utilities']
df_means = []

for df in ohlc_sector_data:
    df_means.append(df.mean())

df_sector = []
i = 0
for df in df_means:
    df = pd.DataFrame(df)
    df['Sector'] = sector_names[i]
    df_sector.append(df)
    i += 1

i = 0
for df in df_sector:
    df.to_csv(f'data/{df_names_avg[i]}.csv', index=True)
    i += 1

- Se define un dataset con todos los promedios de cada simbolo separado por sector.

In [44]:
total_sectors_avgs = pd.concat([df_sector[0], df_sector[1], df_sector[2], df_sector[3], df_sector[4],
                        df_sector[5], df_sector[6], df_sector[7], df_sector[8], df_sector[9],
                        df_sector[10]])
total_sectors_avgs.to_csv(f'data/Total sectors averages.csv', index=True)

In [31]:
df_avgs = pd.DataFrame({'Sectors': [sector_names[0], sector_names[1], sector_names[2], sector_names[3],
                                    sector_names[4], sector_names[5], sector_names[6], sector_names[7],
                                    sector_names[8], sector_names[9], sector_names[10]],
                        'Media': [df_means[0].mean(),df_means[1].mean(),df_means[2].mean(),
                                  df_means[3].mean(),df_means[4].mean(),df_means[5].mean(),
                                  df_means[6].mean(),df_means[7].mean(),df_means[8].mean(),
                                  df_means[9].mean(),df_means[10].mean()]
                        })

In [32]:
df_avgs.to_csv('data/sector_avgs.csv', index=False)